# Disinformation

An example where an individual spreads a fake news among others and they discuss whether it is true or not, as well as some methods of preventing the spread of these.

## Setup

### Import libraries

In [2]:
!pip install sentence-transformers

     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.4 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.4 kB ? eta -:--:--
     -------------------------- ----------- 30.7/44.4 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 44.4/44.4 kB 313.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/255.2 kB ? eta -:--:--
   ---------------------------------------- 255.2/255.2 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/436.6 kB ? eta -:--:--
   --------------------------------------- 436.6/436.6 kB 28.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
    --------------------------------------- 2.7/199.4 MB 57.3 MB/s eta 0:00:04
   - -------------------------------------- 5.1/199.4 MB 53.9 MB/s eta 0:00:04
   - -------------------------------------- 7.6/199.4 MB 54.2 MB/s eta 0:00:04
   -- ------

In [4]:
!pip install termcolor

In [8]:
!pip install absl-py

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)


In [10]:
!pip install reactivex

  Using cached reactivex-4.0.4-py3-none-any.whl.metadata (5.5 kB)
Using cached reactivex-4.0.4-py3-none-any.whl (217 kB)


In [12]:
!pip install openai

  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/383.7 kB ? eta -:--:--
   --------------------------------------- 383.7/383.7 kB 11.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/76.4 kB ? eta -:--:--
   ---------------------------------------- 76.4/76.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/78.0 kB ? eta -:--:--
   ---------------------------------------- 78.0/78.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/199.0 kB ? eta -:--:--
   ---------------------------------------- 199.0/199.0 kB ? eta 0:00:00
Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [13]:
import os
import random
import datetime
import numpy as np
import concurrent.futures
import sentence_transformers

from typing import Callable
from IPython import display

from concordia.clocks import game_clock
from concordia.agents import basic_agent
from concordia.utils import html as html_lib
from concordia.environment import game_master
from concordia.language_model import gpt_model
from concordia.document import interactive_document
from concordia import components as generic_components
from concordia.associative_memory import blank_memories
from concordia.components import agent as agent_components
from concordia.factory.environment import basic_game_master
from concordia.associative_memory import associative_memory
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.utils import measurements as measurements_lib
from concordia.components import game_master as gm_components
from concordia.thought_chains import thought_chains as thought_chains_lib
from concordia.environment.scenes import conversation as conversation_scene

### Language Model Initialization

In [14]:
GPT_API_KEY = 'lm-studio' #@param {type: 'string'}
GPT_MODEL_NAME = 'LM Studio Community/Meta-Llama-3-8B-Instruct-GGUF' #@param {type: 'string'}

if not GPT_API_KEY:
  raise ValueError('GPT_API_KEY is required.')

model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY,
                                   model_name=GPT_MODEL_NAME)

### Sentence Encoder Setup


In [15]:
st_model = sentence_transformers.SentenceTransformer(
    'sentence-transformers/all-mpnet-base-v2')
embedder = lambda x: st_model.encode(x, show_progress_bar=False)

## Configuring the generic knowledge of players and GM

### Define the premise, subgoal (agents' goal), goal, and context

In [16]:
PROJECT_PREMISE = (
    ' The study of disinformation and its rapid spread poses new challenges'
    ' beyond the established conventions of traditional media studies. Unlike'
    ' misinformation, disinformation involves the intentional spread of false'
    ' or misleading information to manipulate. Understanding how fake news'
    ' spreads and evaluating the effectiveness of various strategies to prevent'
    ' its spread is crucial in today´s information landscape.'
)

PROJECT_SUBGOAL = (
    ' Discuss the veracity of the information you receive. Evaluate and share your thoughts on '
    ' whether the news might be true or false, and explain your reasoning. Consider the source, '
    ' your own knowledge, and any available verification tools. '
)

PROJECT_GOAL = (
    ' An agent introduces a piece of fake news and you need to observe its spread. '
    ' Observe the effect it has on each character and assess the effectiveness '
    ' of different strategies and tools used to prevent the spread of fake news,'
    ' providing insights into which methods are most successful.'
)

PROJECT_CONTEXT = (
    ' A group of friends is having a casual conversation when one of them suddenly '
    ' shares a surprising piece of news. The friends then discuss whether the news '
    ' is true or false, evaluating the information based on their knowledge, the '
    ' credibility of the source, and any verification tools they have. This setting '
    ' provides a realistic scenario for observing the spread of fake news and testing '
    ' strategies to prevent it.'
)

In [17]:
agent_goal = PROJECT_SUBGOAL

### Generic memories are memories that all players and GM share.

In [18]:
simulation_premise_component = generic_components.constant.ConstantComponent(
    state=PROJECT_CONTEXT,
    name='The context of the current situation',
)

importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

### Make the clock

In [19]:
UPDATE_INTERVAL = datetime.timedelta(seconds=10)

SETUP_TIME = datetime.datetime(hour=8, year=2024, month=6, day=1)

START_TIME = datetime.datetime(hour=14, year=2024, month=7, day=1)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[UPDATE_INTERVAL, datetime.timedelta(seconds=10)])

NUM_ROUNDS = 3

## Functions to build the agents

In [20]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

In [21]:
from collections.abc import Sequence
from typing import Any
import dataclasses

@dataclasses.dataclass(frozen=True, kw_only=True)
class AgentConfig:
  """A card that describes a player.

  Attributes:
    name: name of the agent.
    gender: the gender of the agent.
    traits: any traits to use while generating formative memories.
    political_ideology: the set of beliefs about political values and the role of government that influence the agent's behavior and decisions.
    biography: a description of the agent.
    context: agent formative memories will be generated with this context
    specific_memories: inject these specific memories. Split memories at newline
      characters. Can be left blank if not used.
    goal: defines agents goal. Can be left blank if not used.
  """

  name: str
  gender: str
  traits: str
  political_ideology: str
  biography: str = ''
  goal: str = ''
  selected_works : Sequence[str]

def specific_memories_from_selected_works(player_config: AgentConfig) -> str:
  """Create memories per player using their role as moderator or debater."""
  specific_memories = []
  player_name = player_config.name
  for work in player_config.selected_works:
    specific_memories += [(
        f'[writing] of {player_name}: "{work}"')]
    idea = model.sample_text(
        (f'Consider the paper summarised here: "{work}". Without repeating '
         'the title, a two sentence TLDR of its most important and distinctive '
         'idea is that '),
        terminators=('\n',))
    specific_memories += [(
        f'[idea] of {player_name}: {idea}')]
    print(f'idea: {idea}.')

  return specific_memories

In [22]:
def cat_with_dropout(inputs : list[str], dropout_rate: float = 0.2) -> str:
  result = '\n'.join([x for x in inputs if random.random() > dropout_rate])
  return result

class MindStream(agent_components.observation.Observation):
  """The MindStream class defines an observation component for an agent, retrieving and logging memories within a specific timeframe"""
  def state(self):
    mems = self._memory.retrieve_time_interval(
        self._clock_now() - self._timeframe, self._clock_now(), add_time=True
    )

    if self._verbose:
      self._log('\n'.join(mems) + '\n')
    return '\n'.join(mems) + '\n'


In [29]:
def build_agent(
    agent_config,
    unused_player_names: list[str],
    unused_measurements: measurements_lib.Measurements | None = None,
):
  """This function builds an agent using the provided configuration. It initializes the agent's memory,
    biography, traits, observations, and other components, and then creates an instance of BasicAgent."""

  agent_name = agent_config.name
  mem = blank_memory_factory.make_blank_memory()

  memories_from_work = specific_memories_from_selected_works(agent_config)

  for item in memories_from_work:
    if item:
      mem.add(item, importance=1.0)

  bio = generic_components.constant.ConstantComponent(
      state=agent_config.biography, name='biography'
  )
  time = generic_components.report_function.ReportFunction(
      name='Current time',
      function=clock.current_time_interval_str,
  )
  traits = generic_components.constant.ConstantComponent(
      state=agent_config.traits, name='psychological traits'
  )
  political_ideology = generic_components.constant.ConstantComponent(
      state=agent_config.political_ideology, name='political ideology'
  )
  current_obs = MindStream(
      agent_name=agent_name,
      clock_now=clock.now,
      memory=mem,
      timeframe=clock.get_step_size()*2,
      component_name='current observations',
  )

  convo_so_far = generic_components.report_function.ReportFunction(
    name='memory of the conversation',
    function=lambda: cat_with_dropout(mem.retrieve_by_regex(r' -- "')),
    )
  
  ideas = generic_components.report_function.ReportFunction(
        name='ideas',
        function=lambda: cat_with_dropout(mem.retrieve_by_regex(r'\[idea\]')),
        )
  
  writing = generic_components.report_function.ReportFunction(
        name='writings',
        function=lambda: cat_with_dropout(mem.retrieve_by_regex(r'\[writing\]')),
        
        )


  # Setup the reflection component and its related components.
  topic_of_debate = generic_components.report_function.ReportFunction(
      name='Topic of the workshop', function=lambda: PROJECT_PREMISE
  )
  goal_of_debate = generic_components.report_function.ReportFunction(
      name='Goal of the workshop', function=agent_goal
  )
  # The agent's subpersonal intuition contains a bias toward thinking that
  # they themselves are the best.

  reflection = agent_components.creative_reflection.CreativeReflection(
      name='reflection',
      model=model,
      memory=mem,
      agent_name=agent_name,
      source_of_abstraction=[convo_so_far, writing, ideas],
      topic_component = topic_of_debate,
      clock_now=clock.now,
      verbose=False,
  )
  all_components = [
          bio,
          traits,
          political_ideology,
          topic_of_debate,
          goal_of_debate,
          reflection,
          current_obs]
  agent = basic_agent.BasicAgent(
      model,
      agent_name=agent_name,
      clock=clock,
      components=[time] +  all_components,
      update_interval=UPDATE_INTERVAL,
  )

  return agent, mem

## Configure and build the agents

In [30]:
# Define a function to generate random traits for an agent
TRAIT_LEVELS = ["low", "medium", "high"] # range of possible levels for traits

def make_random_traits() -> str:
  return str({
      "extraversion": np.random.choice(TRAIT_LEVELS),
      "neuroticism": np.random.choice(TRAIT_LEVELS),
      "openness": np.random.choice(TRAIT_LEVELS),
      "conscientiousness": np.random.choice(TRAIT_LEVELS),
      "agreeableness": np.random.choice(TRAIT_LEVELS),
      "susceptibility": np.random.choice(TRAIT_LEVELS),
      "critical thinking": np.random.choice(TRAIT_LEVELS),
  })


# Create player configurations, each represented by an AgentConfig object
player_configs = [
    AgentConfig(
        name="Alice Johnson",
        gender="female",
        traits=make_random_traits(),
        political_ideology="liberal",
        biography=(
            "Alice Johnson, a political activist and community organizer, is known for her "
            "advocacy on social justice and environmental issues. With a background in sociology, "
            "Alice has spent years working with grassroots organizations to promote equality and "
            "sustainable development. Her liberal views are deeply rooted in a belief in the power "
            "of community action and progressive policies to drive change."
        ),
        selected_works=[
            (
                "The Power of Community: A guide to grassroots organizing and the impact of collective action "
                "on local and national politics. This work emphasizes the importance of community engagement and "
                "advocacy in promoting social justice and environmental sustainability."
            ),
            (
                "Breaking the Chains: Exploring the intersection of social justice and environmental advocacy, "
                "this book discusses the role of policy in addressing systemic inequalities and environmental degradation."
            ),
        ],
    ),
    AgentConfig(
        name="David Smith",
        gender="male",
        traits=make_random_traits(),
        political_ideology="conservative",
        biography=(
            "David Smith is a business executive and political commentator known for his conservative views. "
            "With a career spanning over two decades in the corporate world, David emphasizes the importance "
            "of free markets, personal responsibility, and limited government. His commentaries often critique "
            "progressive policies, advocating instead for traditional values and economic conservatism."
        ),
        selected_works=[
            (
                "The Free Market Solution: A comprehensive analysis of how free market principles can address "
                "economic and social issues more effectively than government intervention. David argues for "
                "reducing regulations and promoting entrepreneurship."
            ),
            (
                "Preserving Our Values: In this book, David explores the importance of maintaining traditional values "
                "and cultural heritage in the face of modern challenges. He discusses the role of family, faith, and community "
                "in fostering a stable and prosperous society."
            ),
        ],
    ),
    AgentConfig(
        name="Emma Thompson",
        gender="female",
        traits=make_random_traits(),
        political_ideology="moderate",
        biography=(
            "Emma Thompson, a journalist and independent political analyst, is known for her balanced and pragmatic approach "
            "to political issues. Emma's work focuses on bridging the gap between polarized political ideologies, advocating for "
            "policies that are practical and beneficial for the majority. Her moderate stance often emphasizes the need for "
            "compromise and bipartisanship in policymaking."
        ),
        selected_works=[
            (
                "Finding Common Ground: This book explores the art of compromise in politics, highlighting successful case studies "
                "where bipartisan efforts have led to meaningful change. Emma discusses the importance of dialogue and understanding "
                "in resolving political conflicts."
            ),
            (
                "The Pragmatic Approach: Emma provides an in-depth analysis of various political issues from a moderate perspective, "
                "emphasizing solutions that are effective and realistic, rather than ideologically driven."
            ),
        ],
    ),
    AgentConfig(
        name="Michael Chen",
        gender="male",
        traits=make_random_traits(),
        political_ideology="libertarian",
        biography=(
            "Michael Chen is a technology entrepreneur and libertarian advocate who champions individual freedom and minimal government "
            "intervention. With a background in software development and startups, Michael believes in the power of innovation and personal "
            "initiative to drive societal progress. His libertarian views are reflected in his emphasis on privacy, free speech, and market "
            "deregulation."
        ),
        selected_works=[
            (
                "The Future of Innovation: Michael discusses how minimal government intervention and maximum personal freedom can spur technological "
                "advancements and economic growth. He explores case studies of successful startups that thrived in deregulated environments."
            ),
            (
                "Freedom and Privacy: This work delves into the importance of protecting individual rights in the digital age. Michael examines the threats "
                "to privacy and free speech posed by government surveillance and corporate control, advocating for robust protections and personal autonomy."
            ),
        ],
    ),
]

In [31]:
NUM_PLAYERS = len(player_configs)
player_names = [player.name for player in player_configs][:NUM_PLAYERS]
measurements = measurements_lib.Measurements()

players = []
memories = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent, mem in pool.map(build_agent,
                             player_configs[:NUM_PLAYERS],
                             # All players get the same `player_names`.
                             [player_names] * NUM_PLAYERS,
                             # All players get the same `measurements` object.
                             [measurements] * NUM_PLAYERS):
    players.append(agent)
    memories[agent.name] = mem

idea: the key to successful politics lies not in confrontation or polarization, but rather in fostering open dialogue and finding common ground between opposing parties through compromise. By doing so, meaningful change can be achieved, as demonstrated by real-world case studies showcasing the positive outcomes of bipartisan efforts..
idea: Minimal government intervention and maximum personal freedom are essential for fostering innovation and driving economic growth by allowing entrepreneurs to take risks and pursue new ideas without regulatory burdens. The paper highlights successful startups that flourished in deregulated environments as examples of how this approach can lead to significant technological advancements and prosperity..
idea: grassroots organizing and collective action have the power to drive significant change at both local and national levels by leveraging the strengths of community engagement and advocacy. By building strong, inclusive communities and amplifying marg

C:\Users\algar\Documents\Travail\GitHub\ConcordiaSims\concordia\associative_memory\associative_memory.py:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(


idea: the author argues that government surveillance and corporate control pose significant threats to individual privacy and freedom, and that strong legal protections are necessary to safeguard these fundamental rights in the digital age. By advocating for robust protections and personal autonomy, the author seeks to empower individuals to exercise their rights and maintain control over their own data and online activities..


C:\Users\algar\Documents\Travail\GitHub\ConcordiaSims\concordia\associative_memory\associative_memory.py:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(


idea: The book argues that environmental degradation and social injustice are interconnected issues that require a holistic approach to address, highlighting the need for policymakers to prioritize policies that simultaneously promote social justice and environmental sustainability. By recognizing the systemic inequalities perpetuated by environmental degradation, the book advocates for a more equitable distribution of resources and benefits, ultimately seeking to "break the chains" of oppression and create a more just and sustainable future..


C:\Users\algar\Documents\Travail\GitHub\ConcordiaSims\concordia\associative_memory\associative_memory.py:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(


idea: the preservation of traditional values and cultural heritage is crucial for building a stable and prosperous society, and this can be achieved through the support of family, faith, and community. By maintaining these core elements, individuals can navigate modern challenges while staying true to their roots and preserving their sense of identity..


C:\Users\algar\Documents\Travail\GitHub\ConcordiaSims\concordia\associative_memory\associative_memory.py:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(


TypeError: Observation.__init__() got an unexpected keyword argument 'agent_name'

### Summarise the perspective of each player

In [ ]:
player_logs = []
player_log_names = []
for player in players:
  name = player.name
  detailed_story = '\n'.join(memories[player.name].retrieve_recent(
      k=1000, add_time=True))
  summary = player.state().splitlines()

  all_player_mem = memories[player.name].retrieve_recent(k=1000, add_time=True)
  all_player_mem = ['Player state:', summary, 'Memories:'] + all_player_mem
  player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
  player_logs.append(player_html)
  player_log_names.append(f'{name}')

tabbed_html = html_lib.combine_html_pages(
    player_logs,
    player_log_names,
    summary='',
    title='Backstory of the players',
)

tabbed_html = html_lib.finalise_html(tabbed_html)
display.HTML(tabbed_html)

## Build GM

In [ ]:
call_to_speech = (
    'Given the above, generate what {name} would say next? Take their '
    'ideas and reflections and the goal of the workship into account. Respond '
    'in the format `{name} -- "..."`'
)

In [ ]:
clock.advance()

debate_event = (
    f'{players[0].name}, {players[1].name}, {players[2].name}, and {players[3].name} are in a'
    f' workshop.\n Their current goal is {agent_goal}.'
)

for player in players:
  player.observe(debate_event)

for player in players:
  player.observe('It is time to for a discussion now')


convo_scene = conversation_scene.make_conversation_game_master(
    players,
    clock=clock,
    model=model,
    memory_factory=blank_memory_factory,
    name='Disinformation',
    premise=debate_event,
    call_to_speech=call_to_speech,
    review_participants=True,
    check_for_termination=False,
    randomise_initiative=True,
)
with clock.higher_gear():
  clock.advance()
  output = convo_scene.run_episode(10)

first_convo_html = html_lib.PythonObjectToHTMLConverter(
    convo_scene.get_history()
).convert()

In [ ]:
display.HTML(first_convo_html)

In [ ]:
essays = []
for player in players:
  prompt = interactive_document.InteractiveDocument(model)
  prompt.statement(player.state())
  agent_name = player.name
  result = prompt.open_question(
      'Generate an essay on the topic of the workshop from the perspective of'
      f' {agent_name}. The goal of the essay is to summarise the conversation'
      f' and {agent_goal}. Write in the style of {agent_name}, taking their'
      ' ideas and reflections into account. Format the output as html',

      max_tokens=5000,
      terminators=(),
  )
  essays.append(result)

tabbed_html = html_lib.combine_html_pages(
    essays,
    [player.name for player in players],
    summary='',
    title='First essays by participants',
)

first_essays_html = html_lib.finalise_html(tabbed_html)
display.HTML(first_essays_html)

### Game Master Perspective

In [ ]:
game_master_memory = associative_memory.AssociativeMemory(
    sentence_embedder=embedder,
    importance=importance_model.importance,
    clock=clock.now)
primary_environment, game_master_memory = (
    basic_game_master.build_game_master(
        model=model,
        embedder=embedder,
        importance_model=importance_model_gm,
        clock=clock,
        players=players,
        shared_memories=[f'{PROJECT_PREMISE}\n{PROJECT_CONTEXT}'],
        shared_context=f'{PROJECT_GOAL}',
        blank_memory_factory=blank_memory_factory,
        memory=game_master_memory,
    )
)

In [ ]:
episode_length = 4  # @param {type: 'integer'}
for _ in range(episode_length):
  primary_environment.step()

In [ ]:
results_html = basic_game_master.create_html_log(
    model=model,
    primary_environment=primary_environment,
    secondary_environments=[],
)

In [ ]:
display.HTML(results_html)